<a href="https://colab.research.google.com/github/yajima-yasutoshi/Model/blob/main/20250702/%E6%9C%80%E7%9F%AD%E8%B7%AF%E5%95%8F%E9%A1%8C%E3%81%AE%E6%BC%94%E7%BF%92%E5%95%8F%E9%A1%8C%E3%81%AE%E8%A7%A3%E8%AA%AC%E3%81%A8%E8%A7%A3%E7%AD%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#最短路問題

### **準備: `python-mip` のインストール**


In [1]:
%%capture
!pip install mip

---

## 演習問題1

### 1. 問題
以下のネットワークについて、ノードSからノードGへの最短経路のコストを、`python-mip` を用いて求めよ。

* **ノード**: {S, A, B, C, D, G}
* **枝 (コスト)**:
    * (S, A, 1), (S, B, 4)
    * (A, B, 2), (A, C, 1), (A, D, 3)
    * (B, D, 1)
    * (C, G, 2)
    * (D, C, 1), (D, G, 6)
* **始点**: S
* **終点**: G

### 2. 数理モデルの定式化
本問題は、与えられたネットワーク上の始点Sから終点Gへの最短路を求める問題であり、0-1整数計画問題として以下のように定式化できる。

* **パラメータ**:
  * $V = \{S, A, B, C, D, G\}$: ノードの集合
  * $E$: 枝の集合 (上記リスト)
  * $s = S$: 始点ノード
  * $t = G$: 終点ノード
  * $c_{ij}$: 枝 $(i,j) \in E$ のコスト

* **決定変数**:
  * $x_{ij} \in \{0, 1\}$: 枝 $(i,j) \in E$ を経路に含める場合に1、そうでない場合に0をとるバイナリ変数。

* **目的関数**:
    経路の総コストを最小化する。
    $$
    \text{Minimize} \quad Z = \sum_{(i,j) \in E} c_{ij} x_{ij}
    $$

* **制約条件**:
    1.  **フロー保存則**:
        * 始点Sからは、ちょうど1つの枝が出ていく。
            $$
            \sum_{j:(S,j) \in E} x_{Sj}  = 1
            $$
        * 終点Gへは、ちょうど1つの枝が入ってくる。
            $$
            \sum_{i:(i,G) \in E} x_{iG} = 1
            $$
        * 中間ノード（S, G以外）では、入ってくる枝の数と出ていく枝の数は等しい。
            $$
            \sum_{j:(k,j) \in E} x_{kj} - \sum_{i:(i,k) \in E} x_{ik} = 0 \quad (\forall k \in V \setminus \{S, G\})
            $$

### 3. Python (MIP) による実装
`python-mip` を用いてこの問題を解くコードは以下の通りである。

In [2]:
from mip import Model, xsum, MINIMIZE, BINARY, OptimizationStatus

# 1. データの定義
# 枝 (始点, 終点, コスト)
edges = [
    ('S', 'A', 1), ('S', 'B', 4),
    ('A', 'B', 2), ('A', 'C', 1), ('A', 'D', 3),
    ('B', 'D', 1),
    ('C', 'G', 2),
    ('D', 'C', 1), ('D', 'G', 6),
]

# ノードの集合を特定
nodes = set()
for u, v, c in edges:
    nodes.add(u)
    nodes.add(v)

# 始点と終点
start_node = 'S'
end_node = 'G'

# 2. 数理モデルの構築
# モデルの作成
model = Model(name="shortest_path_1", sense=MINIMIZE)

# 変数の定義 (x[u,v]は枝(u,v)を使うかどうかのバイナリ変数)
x = { (u, v): model.add_var(name=f"x_{u}_{v}", var_type=BINARY) for u, v, c in edges }

# 目的関数の設定
model.objective = xsum(c * x[u, v] for u, v, c in edges)

# 制約条件の追加 (フロー保存則)
for i in nodes:
    # ノードiから出ていくフローの合計
    flow_out = xsum(x[u, v] for u, v, c in edges if u == i)
    # ノードiへ入ってくるフローの合計
    flow_in = xsum(x[u, v] for u, v, c in edges if v == i)

    if i == start_node:
        model += (flow_out - flow_in == 1, f"flow_conservation_{i}")
    elif i == end_node:
        model += (flow_out - flow_in == -1, f"flow_conservation_{i}")
    else:
        model += (flow_out - flow_in == 0, f"flow_conservation_{i}")

# 3. モデルの求解
status = model.optimize()

# 4. 結果の表示
print("--- 演習問題1 結果 ---")
if status == OptimizationStatus.OPTIMAL:
    print(f"最適解が見つかりました。")
    print(f"最短経路のコスト: {model.objective_value}")
    print("選択された経路:")
    for u, v, c in edges:
        if x[u, v].x >= 0.99: # ほぼ1であれば選択された枝
            print(f"  枝({u}, {v}) コスト: {c}")
else:
    print(f"最適解は見つかりませんでした。ステータス: {status}")

--- 演習問題1 結果 ---
最適解が見つかりました。
最短経路のコスト: 4.0
選択された経路:
  枝(S, A) コスト: 1
  枝(A, C) コスト: 1
  枝(C, G) コスト: 2


---

## 演習問題2

### 1. 問題
以下の都市間の道路網を考える。道路には一方通行のものと両方通行のものがあり、それぞれ異なるコスト(所要時間)がかかる。都市1から都市5への最短コスト (所要時間)を求めよ。

* **ノード**: {1, 2, 3, 4, 5}
* **枝 (始点, 終点, コスト)**:
    * (1, 2, 10) 両方向
    * (1, 3, 15) 両方向
    * (2, 3, 5)  2から3への一方通行
    * (2, 4, 20) 両方向
    * (3, 4, 5)  両方向
    * (3, 5, 25) 3から5への一方通行
    * (4, 5, 10) 両方向
* **始点**: 1
* **終点**: 5

### 2. 数理モデルの定式化
基本的な定式化は演習問題1と同様である。ただし、両方向の枝は2つの独立した一方通行の枝としてモデルに組み込む必要がある。

* **パラメータ**:
  * $V = \{1, 2, 3, 4, 5\}$: ノードの集合
  * $E$: 枝の集合。両方向の枝は2つの枝として表現する。例えば、(1, 2, 10)両方向は、(1, 2, 10)と(2, 1, 10)の2つの枝を意味する。
  * $s = 1$: 始点ノード
  * $t = 5$: 終点ノード
  * $c_{ij}$: 枝 $(i,j) \in E$ のコスト

* **決定変数、目的関数、制約条件**:
    演習問題1の定式化と同じである。

### 3. Python (MIP) による実装
両方向の枝を2つの片方向の枝に展開してデータを準備し、モデルを構築する。

In [3]:
from mip import Model, xsum, MINIMIZE, BINARY, OptimizationStatus

# 1. データの定義
# 一方通行および両方向の枝のリスト
temp_edges = [
    (1, 2, 10, 'both'), (1, 3, 15, 'both'),
    (2, 3, 5, 'one-way'),
    (2, 4, 20, 'both'),
    (3, 4, 5, 'both'),
    (3, 5, 25, 'one-way'),
    (4, 5, 10, 'both')
]

# モデルで扱う枝のリストを作成 (両方向を展開)
edges = []
for u, v, c, direction in temp_edges:
    edges.append((u, v, c))
    if direction == 'both':
        edges.append((v, u, c))

# ノードの集合を特定
nodes = set()
for u, v, c in edges:
    nodes.add(u)
    nodes.add(v)

# 始点と終点
start_node = 1
end_node = 5

# 2. 数理モデルの構築
model = Model(name="shortest_path_2", sense=MINIMIZE)
x = { (u, v): model.add_var(name=f"x_{u}_{v}", var_type=BINARY) for u, v, c in edges }
model.objective = xsum(c * x[u, v] for u, v, c in edges)

for i in nodes:
    flow_out = xsum(x[u, v] for u, v, c in edges if u == i)
    flow_in = xsum(x[u, v] for u, v, c in edges if v == i)
    if i == start_node:
        model += (flow_out - flow_in == 1, f"flow_conservation_{i}")
    elif i == end_node:
        model += (flow_out - flow_in == -1, f"flow_conservation_{i}")
    else:
        model += (flow_out - flow_in == 0, f"flow_conservation_{i}")

# 3. モデルの求解
status = model.optimize()

# 4. 結果の表示
print("--- 演習問題2 結果 ---")
if status == OptimizationStatus.OPTIMAL:
    print(f"最適解が見つかりました。")
    print(f"最短経路のコスト: {model.objective_value}")
    print("選択された経路:")
    for u, v, c in edges:
        # 重複する枝（逆方向）の表示を避けるための簡単なチェック
        if (u,v) in x and x[u, v].x >= 0.99:
            print(f"  枝({u}, {v}) コスト: {c}")
else:
    print(f"最適解は見つかりませんでした。ステータス: {status}")

--- 演習問題2 結果 ---
最適解が見つかりました。
最短経路のコスト: 30.0
選択された経路:
  枝(1, 3) コスト: 15
  枝(3, 4) コスト: 5
  枝(4, 5) コスト: 10


---

## 演習問題3

### 1. 問題
演習問題1のネットワークにおいて、ノードSからノードGへ行く際に、**必ずノードDを経由する**という条件下での最短経路のコストを求めよ。

**考え方**: この問題は2つの独立した最短路問題として解くことができる。
1.  SからDへの最短路
2.  DからGへの最短路
そして、これらの結果を組み合わせる。

### 2. 数理モデルの定式化
この問題は、2つのサブ問題に分割して解く。

* **サブ問題1: SからDへの最短路**
    * 始点: S, 終点: D
    * 目的関数: SからDへの経路コストを最小化
    * 制約条件: 演習問題1と同様のフロー保存則を、Sを始点、Dを終点として適用

* **サブ問題2: DからGへの最短路**
    * 始点: D, 終点: G
    * 目的関数: DからGへの経路コストを最小化
    * 制約条件: 演習問題1と同様のフロー保存則を、Dを始点、Gを終点として適用

最終的な総コストは、サブ問題1の最適値とサブ問題2の最適値の和となる。

### 3. Python (MIP) による実装
まずSからDへの最短路を解き、次にDからGへの最短路を解き、結果を合計する。

In [4]:
from mip import Model, xsum, MINIMIZE, BINARY, OptimizationStatus

# --- サブ問題1: S から D への最短路 ---

# 1. データの定義
edges = [
    ('S', 'A', 1), ('S', 'B', 4), ('A', 'B', 2), ('A', 'C', 1),
    ('A', 'D', 3), ('B', 'D', 1), ('C', 'G', 2), ('D', 'C', 1), ('D', 'G', 6),
]
nodes = set(u for u, v, c in edges) | set(v for u, v, c in edges)
start_node_1 = 'S'
end_node_1 = 'D'

# 2. モデル構築 (S -> D)
model1 = Model(name="S_to_D", sense=MINIMIZE)
x1 = { (u, v): model1.add_var(name=f"x1_{u}_{v}", var_type=BINARY) for u, v, c in edges }
model1.objective = xsum(c * x1[u, v] for u, v, c in edges)

for i in nodes:
    flow_out = xsum(x1[u, v] for u, v, c in edges if u == i)
    flow_in = xsum(x1[u, v] for u, v, c in edges if v == i)
    if i == start_node_1:
        model1 += (flow_out - flow_in == 1)
    elif i == end_node_1:
        model1 += (flow_out - flow_in == -1)
    else:
        model1 += (flow_out - flow_in == 0)

# 3. 求解 (S -> D)
status1 = model1.optimize()

# --- サブ問題2: D から G への最短路 ---

# 1. データの定義 (共通のグラフを使用)
start_node_2 = 'D'
end_node_2 = 'G'

# 2. モデル構築 (D -> G)
model2 = Model(name="D_to_G", sense=MINIMIZE)
x2 = { (u, v): model2.add_var(name=f"x2_{u}_{v}", var_type=BINARY) for u, v, c in edges }
model2.objective = xsum(c * x2[u, v] for u, v, c in edges)

for i in nodes:
    flow_out = xsum(x2[u, v] for u, v, c in edges if u == i)
    flow_in = xsum(x2[u, v] for u, v, c in edges if v == i)
    if i == start_node_2:
        model2 += (flow_out - flow_in == 1)
    elif i == end_node_2:
        model2 += (flow_out - flow_in == -1)
    else:
        model2 += (flow_out - flow_in == 0)

# 3. 求解 (D -> G)
status2 = model2.optimize()

# 4. 結果の表示
print("--- 演習問題3 結果 ---")
if status1 == OptimizationStatus.OPTIMAL and status2 == OptimizationStatus.OPTIMAL:
    print("両方の区間で最適解が見つかりました。")
    total_cost = model1.objective_value + model2.objective_value
    print(f"\nSからDへの最短コスト: {model1.objective_value}")
    print("S->D 経路:")
    for u, v, c in edges:
        if x1[u,v].x >= 0.99: print(f"  枝({u}, {v})")

    print(f"\nDからGへの最短コスト: {model2.objective_value}")
    print("D->G 経路:")
    for u, v, c in edges:
        if x2[u,v].x >= 0.99: print(f"  枝({u}, {v})")

    print(f"\n総最短経路コスト (D経由): {total_cost}")
else:
    print("最適解が見つかりませんでした。")

--- 演習問題3 結果 ---
両方の区間で最適解が見つかりました。

SからDへの最短コスト: 4.0
S->D 経路:
  枝(S, A)
  枝(A, B)
  枝(B, D)

DからGへの最短コスト: 3.0
D->G 経路:
  枝(C, G)
  枝(D, C)

総最短経路コスト (D経由): 7.0


---

## 演習問題4

### 1. 問題
演習問題1のネットワークにおいて、枝 (A,C)のコストが1から10に増加した場合、ノードSからノードGへの最短経路とそのコストはどのように変化するか。変更後のネットワークで最短コストを求めよ。

### 2. 数理モデルの定式化
定式化は演習問題1と全く同じである。唯一の違いは、パラメータ$c_{AC}$の値が1から10に変更される点である。

* **パラメータ**:
    * $c_{AC} = 10$
    * その他のパラメータは演習問題1と同じ。

* **決定変数、目的関数、制約条件**:
    演習問題1の定式化と同じである。

### 3. Python (MIP) による実装
演習問題1のコードを再利用し、枝(A, C)のコスト定義部分のみを変更する。

In [5]:
from mip import Model, xsum, MINIMIZE, BINARY, OptimizationStatus

# 1. データの定義
# 枝(A,C)のコストを10に変更
edges = [
    ('S', 'A', 1), ('S', 'B', 4),
    ('A', 'B', 2), ('A', 'C', 10), # コスト変更
    ('A', 'D', 3), ('B', 'D', 1),
    ('C', 'G', 2), ('D', 'C', 1), ('D', 'G', 6),
]
nodes = set(u for u, v, c in edges) | set(v for u, v, c in edges)
start_node = 'S'
end_node = 'G'

# 2. 数理モデルの構築 (演習問題1と同様)
model = Model(name="shortest_path_4", sense=MINIMIZE)
x = { (u, v): model.add_var(name=f"x_{u}_{v}", var_type=BINARY) for u, v, c in edges }
model.objective = xsum(c * x[u, v] for u, v, c in edges)
for i in nodes:
    flow_out = xsum(x[u, v] for u, v, c in edges if u == i)
    flow_in = xsum(x[u, v] for u, v, c in edges if v == i)
    if i == start_node:
        model += (flow_out - flow_in == 1, f"flow_conservation_{i}")
    elif i == end_node:
        model += (flow_out - flow_in == -1, f"flow_conservation_{i}")
    else:
        model += (flow_out - flow_in == 0, f"flow_conservation_{i}")

# 3. モデルの求解
status = model.optimize()

# 4. 結果の表示
print("--- 演習問題4 結果 ---")
if status == OptimizationStatus.OPTIMAL:
    print(f"最適解が見つかりました。")
    print(f"最短経路のコスト: {model.objective_value}")
    print("選択された経路:")
    for u, v, c in edges:
        if x[u, v].x >= 0.99:
            print(f"  枝({u}, {v}) コスト: {c}")
else:
    print(f"最適解は見つかりませんでした。ステータス: {status}")

--- 演習問題4 結果 ---
最適解が見つかりました。
最短経路のコスト: 7.0
選択された経路:
  枝(S, A) コスト: 1
  枝(A, D) コスト: 3
  枝(C, G) コスト: 2
  枝(D, C) コスト: 1


---

## 演習問題5

### 1. 問題
各枝には通常のコスト(時間)に加えて、「特別料金」が設定されているとする。SからGへの最短時間経路を見つけたいが、支払う「特別料金」の合計が4以内でなければならない。この条件下での最短コスト (時間)を求めよ。

* **枝 (始点, 終点, 時間コスト, 特別料金)**:
    * (S, A, 1, 1)
    * (S, B, 4, 0)
    * (A, B, 2, 1)
    * (A, C, 5, 2)
    * (A, D, 3, 3)
    * (B, D, 1, 3)
    * (C, G, 2, 1)
    * (D, C, 1, 0)
    * (D, G, 6, 2)
* **始点**: S, **終点**: G
* **特別料金上限**: 4

### 2. 数理モデルの定式化
この問題は、標準の最短路問題に新たな制約条件を追加したものである。

* **パラメータ**:
  * $c_{ij}$: 枝 $(i,j) \in E$ の時間コスト
  * $f_{ij}$: 枝 $(i,j) \in E$ の特別料金
  * $F_{max} = 4$: 特別料金の許容上限

* **決定変数**:
    * $x_{ij} \in \{0, 1\}$: 枝 $(i,j) \in E$ を経路に含める場合に1、そうでない場合に0

* **目的関数**:
    総時間コストを最小化する。
    $$
    \text{Minimize} \quad Z = \sum_{(i,j) \in E} c_{ij} x_{ij}
    $$

* **制約条件**:
    1.  **フロー保存則**: 演習問題1と同様。
    2.  **予算制約**: 選択された経路の特別料金の合計は、上限値以下でなければならない。
        $$
        \sum_{(i,j) \in E} f_{ij} x_{ij} \leq F_{max}
        $$

### 3. Python (MIP) による実装
フロー保存則に加えて、特別料金に関する制約を追加する。

In [9]:
from mip import Model, xsum, MINIMIZE, BINARY, OptimizationStatus

# 1. データの定義
# 枝 (始点, 終点, 時間コスト, 特別料金)
edges = [
    ('S', 'A', 1, 1), ('S', 'B', 4, 0), ('A', 'B', 2, 1), ('A', 'C', 5, 2),
    ('A', 'D', 3, 3), ('B', 'D', 1, 3), ('C', 'G', 2, 1), ('D', 'C', 1, 0),
    ('D', 'G', 6, 2)
]
# 時間コストと特別料金を分離して扱う
time_costs = {(u, v): t for u, v, t, f in edges}
special_fees = {(u, v): f for u, v, t, f in edges}
edge_keys = list(time_costs.keys())

nodes = set(u for u, v in edge_keys) | set(v for u, v in edge_keys)
start_node = 'S'
end_node = 'G'
fee_limit = 4

# 2. 数理モデルの構築
model = Model(name="constrained_spp", sense=MINIMIZE)
x = { e: model.add_var(name=f"x_{e[0]}_{e[1]}", var_type=BINARY) for e in edge_keys }

# 目的関数 (時間コストの最小化)
model.objective = xsum(time_costs[e] * x[e] for e in edge_keys)

# 制約条件1: フロー保存則
for i in nodes:
    flow_out = xsum(x[u, v] for u, v in edge_keys if u == i)
    flow_in = xsum(x[u, v] for u, v in edge_keys if v == i)
    if i == start_node:
        model += (flow_out - flow_in == 1)
    elif i == end_node:
        model += (flow_out - flow_in == -1)
    else:
        model += (flow_out - flow_in == 0)

# 制約条件2: 特別料金の上限制約
model += (xsum(special_fees[e] * x[e] for e in edge_keys) <= fee_limit, "fee_constraint")

# 3. モデルの求解
status = model.optimize()

# 4. 結果の表示
print("--- 演習問題5 結果 ---")
if status == OptimizationStatus.OPTIMAL:
    print(f"最適解が見つかりました。")
    print(f"最短時間コスト (料金上限4以内): {model.objective_value}")
    total_fee = sum(special_fees[e] * x[e].x for e in edge_keys)
    print(f"経路の合計特別料金: {total_fee}")
    print("選択された経路:")
    for e in edge_keys:
        if x[e].x >= 0.99:
            print(f"  枝({e[0]}, {e[1]}) 時間: {time_costs[e]}, 料金: {special_fees[e]}")
else:
    print(f"最適解は見つかりませんでした。ステータス: {status}")

--- 演習問題5 結果 ---
最適解が見つかりました。
最短時間コスト (料金上限4以内): 8.0
経路の合計特別料金: 4.0
選択された経路:
  枝(S, A) 時間: 1, 料金: 1
  枝(A, C) 時間: 5, 料金: 2
  枝(C, G) 時間: 2, 料金: 1


---
## 演習問題6

### 1. 問題
ある配送員が始点Xから出発し、目的地Aと目的地Bの両方に荷物を届ける必要がある。荷物を届けた後、始点Xに戻る必要はない。AとBのどちらを先に訪れても良い。始点XからA、AからBの順で訪れる場合の総移動コストと、始点XからB、BからAの順で訪れる場合の総移動コストをそれぞれ計算し、より小さい方の総移動コストを求めよ。

* **ノード**: {X, A, B, C, D}
* **枝(コスト)**:
    (X, A, 5), (X, C, 2), (A, X, 5), (A, B, 3), (A, D, 6), (B, A, 3), (B, D, 2),
    (C, X, 2), (C, A, 4), (C, D, 7), (D, A, 6), (D, B, 2), (D, C, 7)
* **始点**: X
* **目的地**: A, B

**考え方**:
1.  2つの訪問順序 (X→A→B と X→B→A) の総コストを比較する。
2.  そのためには、区間ごとの最短路コストを求める必要がある。
    * 区間1: XからAへの最短路コスト
    * 区間2: AからBへの最短路コスト
    * 区間3: XからBへの最短路コスト
    * 区間4: BからAへの最短路コスト
3.  `コスト(X→A→B) = コスト(X→A) + コスト(A→B)`
4.  `コスト(X→B→A) = コスト(X→B) + コスト(B→A)`
5.  上記2つのコストを比較し、小さい方を最終的な答えとする。

### 2. 数理モデルの定式化
この問題は、必要な区間（X→A, A→B, X→B, B→A）ごとに独立した最短路問題を4回解くことで解決できる。各最短路問題の定式化は演習問題1と同様である。

### 3. Python (MIP) による実装
各区間の最短路問題を解くための汎用的なコードを作成し、始点と終点を変えて複数回実行する。

In [7]:
from mip import Model, xsum, MINIMIZE, BINARY, OptimizationStatus

# 1. 共通データの定義
edges = [
    ('X', 'A', 5), ('X', 'C', 2), ('A', 'X', 5), ('A', 'B', 3), ('A', 'D', 6),
    ('B', 'A', 3), ('B', 'D', 2), ('C', 'X', 2), ('C', 'A', 4), ('C', 'D', 7),
    ('D', 'A', 6), ('D', 'B', 2), ('D', 'C', 7)
]
nodes = set(u for u,v,c in edges) | set(v for u,v,c in edges)

# 最短路を計算するヘルパー的な処理
def solve_spp(start_node, end_node):
    model = Model(sense=MINIMIZE)
    x = { (u, v): model.add_var(var_type=BINARY) for u, v, c in edges }
    model.objective = xsum(c * x[u, v] for u, v, c in edges)

    for i in nodes:
        flow_out = xsum(x[u, v] for u, v, c in edges if u == i)
        flow_in = xsum(x[u, v] for u, v, c in edges if v == i)
        if i == start_node:
            model += (flow_out - flow_in == 1)
        elif i == end_node:
            model += (flow_out - flow_in == -1)
        else:
            model += (flow_out - flow_in == 0)

    model.verbose = 0 # 求解中のログを非表示にする
    status = model.optimize()

    if status == OptimizationStatus.OPTIMAL:
        return model.objective_value
    else:
        return float('inf') # 解が見つからない場合は無限大を返す

# 2. 各区間の最短コストを計算
cost_X_to_A = solve_spp('X', 'A')
cost_A_to_B = solve_spp('A', 'B')
cost_X_to_B = solve_spp('X', 'B')
cost_B_to_A = solve_spp('B', 'A')

# 3. 2つのシナリオの総コストを計算
# シナリオ1: X -> A -> B
total_cost_XAB = cost_X_to_A + cost_A_to_B

# シナリオ2: X -> B -> A
total_cost_XBA = cost_X_to_B + cost_B_to_A

# 4. 結果の表示
print("--- 演習問題6 結果 ---")
print(f"区間 X -> A の最短コスト: {cost_X_to_A}")
print(f"区間 A -> B の最短コスト: {cost_A_to_B}")
print(f"区間 X -> B の最短コスト: {cost_X_to_B}")
print(f"区間 B -> A の最短コスト: {cost_B_to_A}")
print("-" * 20)
print(f"訪問順序 (X -> A -> B) の総コスト: {total_cost_XAB}")
print(f"訪問順序 (X -> B -> A) の総コスト: {total_cost_XBA}")
print("-" * 20)

if total_cost_XAB < total_cost_XBA:
    print(f"より効率的な経路は X -> A -> B で、総コストは {total_cost_XAB} です。")
elif total_cost_XBA < total_cost_XAB:
    print(f"より効率的な経路は X -> B -> A で、総コストは {total_cost_XBA} です。")
else:
    print(f"両方の訪問順序の総コストは同じで、{total_cost_XAB} です。")

--- 演習問題6 結果 ---
区間 X -> A の最短コスト: 5.0
区間 A -> B の最短コスト: 3.0
区間 X -> B の最短コスト: 8.0
区間 B -> A の最短コスト: 3.0
--------------------
訪問順序 (X -> A -> B) の総コスト: 8.0
訪問順序 (X -> B -> A) の総コスト: 11.0
--------------------
より効率的な経路は X -> A -> B で、総コストは 8.0 です。
